# 인터파크

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

- 순위, 책제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

### 1. 데이터 가져오기 

In [2]:
site = 'http://book.interpark.com'
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)
html = req.text

In [3]:
soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려는 데이터의 태그 찾기

In [4]:
lis = soup.select_one('.rankBestContentList ').find_all('li')
len(lis)


65

In [5]:
lis = soup.select('.rankBestContentList > ol > li' )
len(lis)


15

### 3. 하나를 선택해서 원하는 정보추출

- 순위, 제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

- 트렌드 코리아 2022

In [6]:
li = lis[0]
href = li.find('a')['href']
href
# href이 뭘 위한 건지 잘 모르겠다. 



'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354291316&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [7]:
sub_url = site + href 
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354291316&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [8]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [9]:
title = li.select_one('.itemName').get_text()
title


'아직 오지 않은 날들을 위하여 '

In [10]:
author = li.select_one('.author').get_text()
author

'파스칼 브뤼크네르/이세진 역'

In [11]:
pub = li.select_one('.company').get_text()
pub

'인플루엔셜'

In [12]:
price = int(li.select_one('.price').find('em').string.replace(',',''))
price

14400

### 4. 페이지에 있는 전체 데이터 가져오기

- with 제목, 저자, 출판사, 가격 

In [27]:
lines = []
for li in lis:
    
    title = li.select_one('.itemName').get_text()
    author = li.select_one('.author').get_text()
    pub = li.select_one('.company').get_text()
    price = int(li.select_one('.price').find('em').string.replace(',',''))
    
    sub_url = site + href 
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    date = sub_soup.select_one('.bInfo_txt').get_text().split('\n')[3][5:]
    page = int(sub_soup.select_one('.bInfo_txt').get_text().split('\n')[4][5:])
    
    
    lines.append([title, author, pub, price, date, page])

df = pd.DataFrame(lines, columns =['제목', '저자', '출판사', '가격', '출판일', '쪽수'] )
df


,제목,저자,출판사,가격,출판일,쪽수
0,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400,2021년 11월 12일,320
1,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750,2021년 11월 12일,320
2,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300,2021년 11월 12일,320
3,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200,2021년 11월 12일,320
4,흔한남매. 9,"흔한남매, 백난도/유난희 그림",미래엔아이세움,12150,2021년 11월 12일,320
5,나는 장사의 신이다,은현장,떠오름,15120,2021년 11월 12일,320
6,거인의 포트폴리오,강환국,페이지2북스,16200,2021년 11월 12일,320
7,불편한 편의점(15만부 기념 윈터 에디션),김호연,나무옆의자,12600,2021년 11월 12일,320
8,옷소매 붉은 끝동 세트,강미강,청어람,20700,2021년 11월 12일,320
9,"개미 5년, 세후 55억",성현우,모루,15300,2021년 11월 12일,320


# 순위, 발행일, 쪽수(정수) 보류 

In [14]:
lis[0].select_one('.rankBtn_ctrl')['class']     # class 속성 

['rankBtn_ctrl', 'rkNum_B01']

In [ ]:
lis[12].select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [ ]:
lis[12].select('.rankBtn_ctrl')[0]['class']

['rankBtn_ctrl', 'rkNum_M01']

- rank가 한자리

In [25]:
# 1위 
rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]     # [class] : class 다음의 것들을 보여줘
rank

'1'

In [26]:
rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
rank

IndexError: list index out of range

In [ ]:
rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
rank = int(rank)
rank

In [ ]:
date = sub_soup.select_one('.bInfo_txt').get_text().split('\n')[3][5:]
date


'2021년 10월 06일'

In [ ]:
page = int(sub_soup.select_one('.bInfo_txt').get_text().split('\n')[4][5:])
page

452

In [ ]:
rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
rank = int(rank)
rank

15